In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [2]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1))

# define input and load data

In [3]:
product_urlpath = (
    "data/S1B_S6_GRDH_1SDV_20211216T115438_20211216T115501_030050_03968A_0F8A.SAFE/"
)
measurement_group = "S6/VV"
dem_urlpath = "data/Chicago-10m-DEM.tif"
orbit_group = None
calibration_group = None
output_urlpath = "Chicago-10m-GTC-GRD.tif"
correct_radiometry = False
interp_method = "nearest"
multilook = None
grouping_area_factor = (1.0, 1.0)
open_dem_raster_kwargs = {"chunks": {}}
kwargs = {"chunks": 2048}

In [4]:
!ls -d {product_urlpath}
!ls -d {dem_urlpath}

data/S1B_S6_GRDH_1SDV_20211216T115438_20211216T115501_030050_03968A_0F8A.SAFE/
data/Chicago-10m-DEM.tif


In [5]:
orbit_group = orbit_group or f"{measurement_group}/orbit"
calibration_group = calibration_group or f"{measurement_group}/calibration"

measurement_ds = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group, **kwargs)  # type: ignore
measurement = measurement_ds.measurement

dem_raster = scene.open_dem_raster(dem_urlpath, **open_dem_raster_kwargs)

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought

# scene

In [6]:
dem_raster

<xarray.DataArray 'band_data' (y: 2500, x: 2500)>
dask.array<getitem, shape=(2500, 2500), dtype=float32, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05 4e+05
  * y            (y) float64 4.625e+06 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
    spatial_ref  int64 ...

In [7]:
# _ = dem_raster.plot()

In [8]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

CPU times: user 3.12 s, sys: 217 ms, total: 3.34 s
Wall time: 3.34 s


<xarray.DataArray 'dem_3d' (axis: 3, y: 2500, x: 2500)>
dask.array<setitem, shape=(3, 2500, 2500), dtype=float64, chunksize=(3, 2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05 4e+05
    spatial_ref  int64 ...
  * y            (y) float64 4.625e+06 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
  * axis         (axis) int64 0 1 2

# acquisition

In [9]:
measurement

<xarray.DataArray 'measurement' (azimuth_time: 15764, ground_range: 8055)>
dask.array<open_dataset-88be9bb1f231cea73d01dffdeffddf21measurement, shape=(15764, 8055), dtype=float32, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
    pixel         (ground_range) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
    line          (azimuth_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-16T11:54:38.079554 .....
  * ground_range  (ground_range) float64 0.0 10.0 20.0 ... 8.053e+04 8.054e+04
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00150106729952866
    slant_range_time_interval:  2.1313598509794117e-08
    sat:anx_datetime:           2021-12-16T11:16:48.149227Z
    units:                      1
    long_name:                  digital number

In [10]:
%%time
acquisition = apps.simulate_acquisition(position_ecef, dem_ecef)
acquisition

CPU times: user 10.6 s, sys: 3.07 s, total: 13.7 s
Wall time: 5.72 s


<xarray.Dataset>
Dimensions:           (x: 2500, y: 2500, axis: 3)
Coordinates:
  * x                 (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
  * axis              (axis) int64 0 1 2
Data variables:
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    dem_direction     (axis, y, x) float64 dask.array<chunksize=(3, 2500, 2500), meta=np.ndarray>

In [11]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

CPU times: user 7.18 ms, sys: 836 µs, total: 8.01 ms
Wall time: 7.37 ms


<xarray.DataArray (azimuth_time: 15764, ground_range: 8055)>
dask.array<pow, shape=(15764, 8055), dtype=float32, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
    pixel         (ground_range) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
    line          (azimuth_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-16T11:54:38.079554 .....
  * ground_range  (ground_range) float64 0.0 10.0 20.0 ... 8.053e+04 8.054e+04
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00150106729952866
    slant_range_time_interval:  2.1313598509794117e-08
    sat:anx_datetime:           2021-12-16T11:16:48.149227Z
    units:                      m2 m-2
    long_name:                  beta nought

In [12]:
coordinate_conversion = xr.open_dataset(
    product_urlpath,
    engine="sentinel-1",
    group=f"{measurement_group}/coordinate_conversion",
    **kwargs,
)  # type: ignore
ground_range = xarray_sentinel.slant_range_time_to_ground_range(
    acquisition.azimuth_time,
    acquisition.slant_range_time,
    coordinate_conversion,
)
interp_kwargs = {"ground_range": ground_range}
ground_range

<xarray.DataArray (y: 2500, x: 2500)>
dask.array<sum-aggregate, shape=(2500, 2500), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    azimuth_time  (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x             (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05 4e+05
    spatial_ref   int64 0
  * y             (y) float64 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06

In [13]:
%%time
geocoded = apps.interpolate_measurement(
    beta_nought,
    multilook=multilook,
    azimuth_time=acquisition.azimuth_time,
    interp_method=interp_method,
    **interp_kwargs,
)

geocoded

CPU times: user 3.53 s, sys: 984 ms, total: 4.52 s
Wall time: 2.92 s


<xarray.DataArray (y: 2500, x: 2500)>
dask.array<dask_aware_interpnd, shape=(2500, 2500), dtype=float32, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    pixel         (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    line          (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    azimuth_time  (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    ground_range  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x             (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05 4e+05
    spatial_ref   int64 0
  * y             (y) float64 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00150106729952866
    slant_range_time_interval:  2.1313598509794117e-08
    sat:anx_datetime:           2021-12-16T11:16:48.149227Z
    units:                      m2 m-2
    long_name:                  beta nought

In [14]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [15]:
# _ = geocoded.plot(vmax=1.0)